## Analyzing Dialysis Facilities in the United States  
This project will analyze dialysis facilities in the United States and attempt to answer the following question:
 - What factors lead to better care in a dialysis facility?

https://github.com/carl-schick-ds/meteorite-landings

https://catalog.data.gov/dataset/medicare-dialysis-facilities

***
### Setup
Import needed libraries.  Unless otherwise noted, all libraries are available in the baseline conda environment.

In [48]:
# Import Libraries
import pandas as pd
import numpy as np
from IPython.display import display

In [ ]:
# Auto Re-load External Modules
%load_ext autoreload
%autoreload 2

In [ ]:
# Toggle REFRESH_DATA literal

REFRESH_DATA = False

In [ ]:
# Refresh Data
# See the dialysis_facilities.py file in this repostitory for details on the data collection routines

if REFRESH_DATA:
    import dialysis_facilities_dc as dc

    # Get the data
    raw_url = 'https://data.cms.gov/sites/default/files/2021-01/FY_2021_Facility_Level_Dialysis_Facility_Reports.csv'
    raw_facilities_df = pd.read_csv(raw_url, dtype={'NPI': 'str', 'Alternate CCN(s)': 'str'})
    print()
    meteorite_data = dc.get_meteorite_data()
    print()
    meteorite_data = dc.process_locations(meteorite_data)
    print()
    meteorite_data = dc.get_country_data(meteorite_data)

    # Convert to CSV file
    meteorite_data.to_csv('meteorite_landings.csv')

In [49]:
# raw_url = 'https://data.cms.gov/sites/default/files/2021-01/FY_2021_Facility_Level_Dialysis_Facility_Reports.csv'
# raw_facilities_df = pd.read_csv(raw_url, dtype={'NPI': 'str', 'Alternate CCN(s)': 'str'})
raw_facilities_df = pd.read_csv('FY_2021_Facility_Level_Dialysis_Facility_Reports.csv', dtype={'NPI': 'str', 'Alternate CCN(s)': 'str'})
display(raw_facilities_df.head())

,State,CCN,Provider Name,City,Ownership Type,ESRD Network,NPI,Chain Name,Modality,Alternate CCN(s),Measure,Measure Score,Year(s) covered by the measure,Measure ID
0,AL,12306,CHILDRENS HOSPITAL OF ALABAMA ESRD,BIRMINGHAM,Non-profit,8,1720166085,INDEPENDENT,Hemodialysis and Peritoneal Dialysis,"012306,013300",F: Prevalent Patients - End of Year Status: Nu...,20.00,2016,pahy1_f
1,AL,12306,CHILDRENS HOSPITAL OF ALABAMA ESRD,BIRMINGHAM,Non-profit,8,1720166085,INDEPENDENT,Hemodialysis and Peritoneal Dialysis,"012306,013300",F: Prevalent Patients - Age: Average patient a...,8.40,2016,agey1_f
2,AL,12306,CHILDRENS HOSPITAL OF ALABAMA ESRD,BIRMINGHAM,Non-profit,8,1720166085,INDEPENDENT,Hemodialysis and Peritoneal Dialysis,"012306,013300",F: Prevalent Patients - Vintage: Average Years...,3.58,2016,viny1_f
3,AL,12306,CHILDRENS HOSPITAL OF ALABAMA ESRD,BIRMINGHAM,Non-profit,8,1720166085,INDEPENDENT,Hemodialysis and Peritoneal Dialysis,"012306,013300",F: Prevalent Patients - Age: % Less than 18 ye...,100.00,2016,age1y1_f
4,AL,12306,CHILDRENS HOSPITAL OF ALABAMA ESRD,BIRMINGHAM,Non-profit,8,1720166085,INDEPENDENT,Hemodialysis and Peritoneal Dialysis,"012306,013300",F: Prevalent Patients - Age: % Between 18-64 y...,0.00,2016,age2y1_f


In [50]:
# raw_facilities_df.info()
# display(raw_facilities_df['Measure'].nunique())
# display(raw_facilities_df['Measure ID'].nunique())
# display(raw_facilities_df['CCN'].nunique())
# display(raw_facilities_df['NPI'].nunique())
# display(raw_facilities_df['Provider Name'].nunique())
# display(raw_facilities_df['Chain Name'].value_counts())
# npi_list = raw_facilities_df['NPI'].value_counts()
# npi_blank_CCNs = raw_facilities_df[raw_facilities_df['NPI'] == "."]['CCN'].value_counts()
# raw_facilities_df[raw_facilities_df['CCN'] == 332770]

In [51]:
facility_cols = ['State', 'CCN', 'Provider Name', 'City', 'Ownership Type', 'ESRD Network', 'NPI', 'Chain Name', 'Modality', 'Alternate CCN(s)']
facilities_df = raw_facilities_df.drop_duplicates(subset=facility_cols)[facility_cols].set_index('CCN')

print('Raw Facilities:', raw_facilities_df['CCN'].nunique())
print('Extracted Facilities:', facilities_df.shape[0])

Raw Facilities: 7920
Extracted Facilities: 7920


In [52]:
measure_cols = ['Measure', 'Measure ID']
measures_df = raw_facilities_df.drop_duplicates(subset=measure_cols)[measure_cols].set_index('Measure ID')

corr_text_1 = 'F: Adult Incident Patients (2728) - Average age, '
corr_text_2 = 'F: SHR (ED) - % Patients with at Least One ED Visit, '
agmy_corr_dict = {'agemy1_f': corr_text_1 + '2016', 'agemy2_f': corr_text_1 + '2017', 'agemy3_f': corr_text_1 + '2018', 'agemy4_f': corr_text_1 + '2019'}
edpt_corr_dict = {'edpty1_f': corr_text_2 + '2016', 'edpty2_f': corr_text_2 + '2017', 'edpty3_f': corr_text_2 + '2018', 'edpty4_f': corr_text_2 + '2019'}

for key in agmy_corr_dict:
    measures_df.loc[key]['Measure'] = agmy_corr_dict[key]
for key in edpt_corr_dict:
    measures_df.loc[key]['Measure'] = edpt_corr_dict[key]

print('Raw Measures:', raw_facilities_df['Measure ID'].nunique())
print('Extracted Measures:', measures_df.shape[0])

Raw Measures: 2093
Extracted Measures: 2093


In [53]:
# display(raw_facilities_df['Year(s) covered by the measure'].value_counts())
raw_facilities_df.rename(columns={'Year(s) covered by the measure' : 'Year'}, inplace=True)
fac_scores_df = raw_facilities_df[['CCN', 'Measure ID', 'Year', 'Measure Score']]

print('Raw Scores:', raw_facilities_df.shape[0])
print('Extracted Scores:', fac_scores_df.shape[0])

Raw Scores: 9701304
Extracted Scores: 9701304
